#### Smart City project:

This code is just to visualize json files.

In [64]:
import os
import shutil

import matplotlib.pyplot as plt
import numpy as np

import pandas as pd
import seaborn as sns
pd.set_option('display.max_columns', 500)
from IPython.display import display

%matplotlib inline

In [65]:
df = pd.read_json('uwo_sample_tls.json')  #Reading a json file into panda df.

df.shape



(32, 348)

In [73]:
#SensorLog3001-12m.txt

#df = pd.read_csv('SensorLog3001-12m.txt', sep=",", header=None)  #Reading a json file into panda df.
data = np.loadtxt("SensorLog3001-12m.txt",dtype='str',delimiter=',')
#dataset=pd.read_csv("SensorLog3001-12m.txt", header=None, engine='python', sep=',', names=None, 
#                index_col=None, usecols=None, squeeze=False)

#dataset.head()

#print(np.shape(data))

dataset = pd.DataFrame({'House Number': data[:, 0], 'Sensor_num': data[:, 1], 'Status': data[:, 2], 'Day': data[:, 3], 'Month': data[:, 4], 'Year': data[:, 5], 'Hour': data[:, 6], 'Min': data[:, 7], 'Counter': data[:, 8], 'Time_elapsed': data[:, 9]}
                      , columns=['House Number', 'Sensor_num', 'Status','Day','Month', 'Year', 'Hour', 'Min', 'Counter','Time_elapsed' ])

dataset.head(40)

#bed = dataset[Sensor_num='Sensor 5']

bed = dataset[dataset['Sensor_num']=='Sensor 5']

bed.head(20)


,House Number,Sensor_num,Status,Day,Month,Year,Hour,Min,Counter,Time_elapsed
10,Res3001,Sensor 5,Pat's Bedroom Motion Idle,23,10,2017,10,27,627,2
22,Res3001,Sensor 5,Pat's Bedroom Motion Activated,23,10,2017,10,50,650,3
25,Res3001,Sensor 5,Pat's Bedroom Motion Idle,23,10,2017,10,52,652,1
36,Res3001,Sensor 5,Pat's Bedroom Motion Activated,23,10,2017,11,28,688,3
37,Res3001,Sensor 5,Pat's Bedroom Motion Idle,23,10,2017,11,31,691,3
105,Res3001,Sensor 5,Pat's Bedroom Motion Activated,23,10,2017,14,25,865,17
109,Res3001,Sensor 5,Pat's Bedroom Motion Idle,23,10,2017,14,37,877,3
121,Res3001,Sensor 5,Pat's Bedroom Motion Activated,23,10,2017,15,42,942,2
127,Res3001,Sensor 5,Pat's Bedroom Motion Idle,23,10,2017,16,8,968,2
139,Res3001,Sensor 5,Pat's Bedroom Motion Activated,23,10,2017,16,51,1011,2


In [80]:
#bed.head()

stacked = bed[['Status']].stack()
bed.loc[:, ['Status']] = pd.Series(stacked.factorize()[0], index=stacked.index).unstack()


bed.head(20)

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,House Number,Sensor_num,Status,Day,Month,Year,Hour,Min,Counter,Time_elapsed
10,Res3001,Sensor 5,0,23,10,2017,10,27,627,2
22,Res3001,Sensor 5,1,23,10,2017,10,50,650,3
25,Res3001,Sensor 5,0,23,10,2017,10,52,652,1
36,Res3001,Sensor 5,1,23,10,2017,11,28,688,3
37,Res3001,Sensor 5,0,23,10,2017,11,31,691,3
105,Res3001,Sensor 5,1,23,10,2017,14,25,865,17
109,Res3001,Sensor 5,0,23,10,2017,14,37,877,3
121,Res3001,Sensor 5,1,23,10,2017,15,42,942,2
127,Res3001,Sensor 5,0,23,10,2017,16,8,968,2
139,Res3001,Sensor 5,1,23,10,2017,16,51,1011,2
